In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df_courses = pd.read_json('./data/processed_data/course_data_clean.json')

### Here's what the data looks like:

In [3]:
df_courses

,course_code,course_title,num_ratings,useful,easy,liked,reviews,num_reviews,num_reviews_with_rating,good_course
0,CS 115,Introduction to Computer Science 1,2111,21%,10%,23%,[{'review_text': 'go to office hours and pract...,86,83,0
1,MATH 135,Algebra for Honours Mathematics,1186,84%,41%,78%,"[{'review_text': 'Welcome to Waterloo Math.', ...",253,250,1
2,ECON 101,Introduction to Microeconomics,1143,64%,70%,45%,[{'review_text': 'Took it online in W21 during...,214,210,0
3,PSYCH 101,Introductory Psychology,899,73%,67%,79%,"[{'review_text': 'Really easy, the course was ...",8,8,1
4,MATH 137,Calculus 1 for Honours Mathematics,780,86%,56%,69%,"[{'review_text': 'and then isaac newton said ""...",171,167,1
...,...,...,...,...,...,...,...,...,...,...
8474,BE 680,Consulting,0,N/A,N/A,N/A,[],0,0,0
8475,KIN 658,Physical Activity and Cognition,0,N/A,N/A,N/A,[],0,0,0
8476,ERS 620,Skills Identification and Career Development,0,N/A,N/A,N/A,[],0,0,0
8477,KIN 659,Wearable Technology,0,N/A,N/A,N/A,[],0,0,0


### And here's an example of what a review looks like:

In [4]:
df_courses['reviews'].values[0][0]

{'review_text': 'go to office hours and practice',
 'course_rating': 'liked course',
 'course_rating_int': 1}

Quick preprocessing to extract reviews and labels

In [5]:
df_reviews = {'course_code': [], 'review': [], 'label': []}
for course in df_courses[['course_code', 'reviews']].values:
    course_code = course[0]
    reviews = course[1]
    retval = []
    for review in reviews:
        review_text = review['review_text']
        label = review['course_rating_int']
        df_reviews['course_code'].append(course_code)
        df_reviews['review'].append(review_text)
        df_reviews['label'].append(label)
        
df_reviews = pd.DataFrame(df_reviews).dropna(axis=0)

In [6]:
df_reviews

,course_code,review,label
0,CS 115,go to office hours and practice,1.0
1,CS 115,One of my least favourite courses. Although th...,0.0
2,CS 115,It starts with a very low pace but after midte...,0.0
3,CS 115,Took this in 2018 with no programming experien...,1.0
4,CS 115,I loved everything about cs 115. Great instruc...,1.0
...,...,...,...
14830,GSJ 410,This is a really wonderful course! Dr. Fraser ...,1.0
14831,SOC 436,While this course ended up being relatively in...,1.0
14832,BME 588,Personally did not find Computational Biomecha...,0.0
14833,ASL 102R,Took this online with Georgia Whalen. As it wa...,1.0


In [13]:
y = list(df_reviews['label'].values)
oh_y = np.zeros((len(y), 2))
oh_y[np.arange(len(y)), np.array(y).astype(int)] = 1

X_train, X_test, y_train, y_test = train_test_split(list(df_reviews['review'].values), oh_y, test_size=0.33, random_state=6)

## Training

In [14]:
import tensorflow as tf
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForTokenClassification, create_optimizer
from transformers import TFDistilBertForSequenceClassification

2022-09-21 02:18:56.455770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-21 02:18:56.568040: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-21 02:18:57.020798: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-21 02:18:57.020883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

In [15]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [16]:
X_train_tokenized = tokenizer(X_train, return_tensors="tf", padding=True, truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="tf", padding=True, truncation=True)

train_ds = tf.data.Dataset.from_tensor_slices((
    dict(X_train_tokenized),
    y_train
))

test_ds = tf.data.Dataset.from_tensor_slices((
    dict(X_test_tokenized),
    y_test
))

2022-09-21 02:18:59.781915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 02:18:59.782918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 02:18:59.786403: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 02:18:59.787324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 02:18:59.788239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [17]:
# Set hyperparams/constants
model_name = "distilbert-base-uncased"
max_length = 96
epochs = 5
batch_size = 32
learning_rate = 2e-5
weight_decay_rate=0.01
num_train_steps = len(X_train) // batch_size
num_warmup_steps = 0

In [18]:
# trying create_optimizer
optimizer, _ = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=num_warmup_steps
)

In [19]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

2022-09-21 02:19:02.054401: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at disti

In [37]:
import tensorflow_addons as tfa

metric = tfa.metrics.F1Score(num_classes=2, threshold=0.5)

def f1_m(y_true, y_pred):
    return metric(y_true, y_pred)

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=5e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[f1_m]
)

In [38]:
model.predict(tokenizer.encode("how are you",
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf"))

1/1 [==============================] - 1s 950ms/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 0.2220673 , -0.11180453]], dtype=float32), hidden_states=None, attentions=None)

In [39]:
model.fit(
    train_ds.batch(batch_size),
    epochs=epochs,
    validation_data=test_ds.batch(batch_size)
)

Epoch 1/5
306/306 [==============================] - 140s 440ms/step - loss: 0.3168 - f1_m: 0.7825 - val_loss: 0.4220 - val_f1_m: 0.7953
Epoch 2/5
306/306 [==============================] - 134s 439ms/step - loss: 0.2145 - f1_m: 0.8117 - val_loss: 0.5648 - val_f1_m: 0.8239
Epoch 3/5
306/306 [==============================] - 134s 439ms/step - loss: 0.1557 - f1_m: 0.8346 - val_loss: 0.6347 - val_f1_m: 0.8409
Epoch 4/5
306/306 [==============================] - 134s 439ms/step - loss: 0.1063 - f1_m: 0.8476 - val_loss: 0.8923 - val_f1_m: 0.8531
Epoch 5/5
306/306 [==============================] - 134s 439ms/step - loss: 0.0785 - f1_m: 0.8573 - val_loss: 0.8809 - val_f1_m: 0.8620


In [40]:
model.save('./saved_model/tf_distilbert_course_reviews_01.pth')

INFO:tensorflow:Assets written to: ./saved_model/tf_distilbert_course_reviews_01.pth/assets


INFO:tensorflow:Assets written to: ./saved_model/tf_distilbert_course_reviews_01.pth/assets
